In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import pickle

In [2]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/deji/mlruns/1', creation_time=1746445402512, experiment_id='1', last_update_time=1746445402512, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
# def read_dataframe(training_data, validation_data):
#     df_train = pd.read_parquet(training_data)
#     df_val = pd.read_parquet(validation_data)
    
#     df_train["duration"] = df_train.lpep_dropoff_datetime - df_train.lpep_pickup_datetime
#     df_val["duration"] = df_val.lpep_dropoff_datetime - df_val.lpep_pickup_datetime
    
#     #Convert duration to minutes
#     df_train.duration = (df_train.duration.dt.total_seconds() / 60)
#     df_train = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]

#     df_val.duration = (df_val.duration.dt.total_seconds() / 60)
#     df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

#     df_train["PU_DO"] = df_train.PULocationID.astype(str) + '_' + df_train.DOLocationID.astype(str)
#     df_val["PU_DO"] = df_val.PULocationID.astype(str) + '_' + df_val.DOLocationID.astype(str)
    
#     #categorical = ['PULocationID', 'DOLocationID']
#     categorical = ['PU_DO']
#     numerical = ['trip_distance']

#     # PULocationID and DOLocationID are actually categorical, so we must change them from int64 to string
#     df_train[categorical] = df_train[categorical].astype(str)
#     df_val[categorical] = df_val[categorical].astype(str)

#     dv = DictVectorizer()
#     train_dicts = df_train[categorical + numerical].to_dict(orient='records')
#     X_train = dv.fit_transform(train_dicts)

#     val_dicts = df_val[categorical + numerical].to_dict(orient='records')

#     X_val = dv.transform(val_dicts)

#     target = 'duration'
#     y_train = df_train[target].values
#     y_val = df_val[target].values

#     with mlflow.start_run():

#         mlflow.set_tag("developer", "deji")

#         mlflow.log_param("train-data-path", "data/green_tripdata_2021-01.parquet")
#         mlflow.log_param("validation-data-path", "data/green_tripdata_2021-02.parquet")

#         alpha = 0.01
    
#         lr = LinearRegression()
#         lr.fit(X_train, y_train)
    
#         y_pred = lr.predict(X_val)
    
#         sns.distplot(y_pred, label='prediction')
#         sns.distplot(y_val, label='actual')
#         plt.legend()
    
#         rmse = np.sqrt(mean_squared_error(y_val, y_pred))
#         mlflow.log_metric("rmse", rmse)        

#     with open('models/lin_reg.bin', 'wb') as f_out:
#         pickle.dump((dv, lr), f_out) 

#     return plt, rmse
# read_dataframe('data/green_tripdata_2021-01.parquet', 'data/green_tripdata_2021-02.parquet')

In [13]:
def prepare_training_and_validation_matrices(training_data, validation_data):
    df_train = pd.read_parquet(training_data)
    df_val = pd.read_parquet(validation_data)
    
    df_train["duration"] = df_train.lpep_dropoff_datetime - df_train.lpep_pickup_datetime
    df_val["duration"] = df_val.lpep_dropoff_datetime - df_val.lpep_pickup_datetime
    
    #Convert duration to minutes
    df_train.duration = (df_train.duration.dt.total_seconds() / 60)
    df_train = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]

    df_val.duration = (df_val.duration.dt.total_seconds() / 60)
    df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

    df_train["PU_DO"] = df_train.PULocationID.astype(str) + '_' + df_train.DOLocationID.astype(str)
    df_val["PU_DO"] = df_val.PULocationID.astype(str) + '_' + df_val.DOLocationID.astype(str)
    
    #categorical = ['PULocationID', 'DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']

    # PULocationID and DOLocationID are actually categorical, so we must change them from int64 to string
    df_train[categorical] = df_train[categorical].astype(str)
    df_val[categorical] = df_val[categorical].astype(str)

    dv = DictVectorizer()
    train_dicts = df_train[categorical + numerical].to_dict(orient='records')
    X_train = dv.fit_transform(train_dicts)

    val_dicts = df_val[categorical + numerical].to_dict(orient='records')

    X_val = dv.transform(val_dicts)

    target = 'duration'
    y_train = df_train[target].values
    y_val = df_val[target].values

    return X_train, y_train, X_val, y_val, dv

In [14]:
X_train, y_train, X_val, y_val, dv = prepare_training_and_validation_matrices('data/green_tripdata_2021-01.parquet', 'data/green_tripdata_2021-02.parquet')

In [5]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [7]:
with mlflow.start_run():

    mlflow.set_tag("developer", "deji")

    mlflow.log_param("train-data-path", "data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data-path", "data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [6]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [8]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=12,
    trials=Trials()
)

In [ ]:
learning_rate
0.41850165383082605
max_depth
8
min_child_weight
2.3444627564668026
objective
reg:linear
reg_alpha
0.22403382940598554
reg_lambda
0.00757957526520513
seed
42

In [15]:

params = {

'learning_rate': 0.41850165383082605,
'max_depth': 8,
'min_child_weight': 2.3444627564668026,
'objective': 'reg:linear',
'reg_alpha': 0.22403382940598554,
'reg_lambda': 0.00757957526520513,
'seed': 42
    
}

with mlflow.start_run():
    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(params)
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    # Calculate RMSE
    y_pred = booster.predict(valid)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print(f"Validation RMSE: {rmse:.4f}")
    
    # Log the metric
    mlflow.log_metric("rmse", rmse)

    # Log artifact
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    
    # Log the model
    mlflow.xgboost.log_model(booster, "model")
    

/home/deji/mlops-env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [00:39:05] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:9.15249
[1]	validation-rmse:7.76530
[2]	validation-rmse:7.18516
[3]	validation-rmse:6.93460
[4]	validation-rmse:6.82073
[5]	validation-rmse:6.76668
[6]	validation-rmse:6.73407
[7]	validation-rmse:6.71766
[8]	validation-rmse:6.70632
[9]	validation-rmse:6.70118
[10]	validation-rmse:6.69766
[11]	validation-rmse:6.69341
[12]	validation-rmse:6.68847
[13]	validation-rmse:6.68536
[14]	validation-rmse:6.68234
[15]	validation-rmse:6.67995
[16]	validation-rmse:6.67671
[17]	validation-rmse:6.67228
[18]	validation-rmse:6.66875
[19]	validation-rmse:6.66698
[20]	validation-rmse:6.66468
[21]	validation-rmse:6.66140
[22]	validation-rmse:6.65678
[23]	validation-rmse:6.65303
[24]	validation-rmse:6.65089
[25]	validation-rmse:6.64853
[26]	validation-rmse:6.64559
[27]	validation-rmse:6.64328
[28]	validation-rmse:6.63958
[29]	validation-rmse:6.63862
[30]	validation-rmse:6.63631
[31]	validation-rmse:6.63513
[32]	validation-rmse:6.63340
[33]	validation-rmse:6.63085
[34]	validation-rmse:6.6

/home/deji/mlops-env/lib/python3.13/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [00:41:03] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/05/12 00:41:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [16]:
# import mlflow
logged_model = 'runs:/95dd770f546f49288f91245d13f93785/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model

# Predict on a Pandas DataFrame.
# import pandas as pd
# loaded_model.predict(pd.DataFrame(data))

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.xgboost
  run_id: 95dd770f546f49288f91245d13f93785

In [17]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [18]:
xgboost_model

In [19]:
xgboost_model.predict(valid)

array([14.690525 ,  6.9833508, 16.240423 , ..., 13.674216 ,  6.7354736,
        8.182596 ], shape=(61921,), dtype=float32)